In [1]:
using Pkg

In [2]:
Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


In [3]:
using JuMP, HiGHS

In [4]:
m = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [5]:
@variable(m, x >= 0)

x

In [6]:
@variable(m, y >= 0)

y

In [7]:
@objective(m, Max, 3x + 4y)

3 x + 4 y

In [8]:
@constraint(m, x + 2y <= 300)

x + 2 y ≤ 300

In [9]:
@constraint(m, 2x + y <= 300)

2 x + y ≤ 300

In [10]:
JuMP.latex_formulation(m)

$$ \begin{aligned}
\max\quad & 3 x + 4 y\\
\text{Subject to} \quad & x + 2 y \leq 300\\
 & 2 x + y \leq 300\\
 & x \geq 0\\
 & y \geq 0\\
\end{aligned} $$

In [11]:
optimize!(m)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 2 rows; 2 cols; 4 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [3e+00, 4e+00]
  Bound  [0e+00, 0e+00]
  RHS    [3e+02, 3e+02]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.9999931588e+00 Ph1: 2(6); Du: 2(6.99999) 0s
          2     7.0000000000e+02 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  7.0000000000e+02
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


In [12]:
value(x)

100.0

In [13]:
value(y)

100.0

In [14]:
objective_value(m)

700.0

### The solution

$x = 100, y = 100, z = 700$

## Unknown capacities

In [16]:
begin
    m = Model(HiGHS.Optimizer)
    @variable(m, x >= 0)
    @variable(m, y >= 0)
    @variable(m, cap1 >= 0)
    @variable(m, cap2 >= 0)
    @objective(m, Max, 3x + 4y)
    @constraint(m, x + 2y <= cap1)
    @constraint(m, 2x + y <= cap2)
    @constraint(m, cap1 + cap2 == 600)
    optimize!(m)
    println(value.([x, y, cap1, cap2]))
    println(objective_value(m))
end

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 3 rows; 4 cols; 8 nonzeros
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [3e+00, 4e+00]
  Bound  [0e+00, 0e+00]
  RHS    [6e+02, 6e+02]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-1); columns 3(-1); elements 6(-2)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.9999704133e+00 Ph1: 2(6); Du: 2(6.99997) 0s
          3    -8.0000000000e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  8.0000000000e+02
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
[0.0, 200.0, 400.0, 200.0]
800.0


In [17]:
latex_formulation(m)

$$ \begin{aligned}
\max\quad & 3 x + 4 y\\
\text{Subject to} \quad & cap1 + cap2 = 600\\
 & x + 2 y - cap1 \leq 0\\
 & 2 x + y - cap2 \leq 0\\
 & x \geq 0\\
 & y \geq 0\\
 & cap1 \geq 0\\
 & cap2 \geq 0\\
\end{aligned} $$

### New Solution

$x = 0, y = 200, \text{cap1} = 400, \text{cap2} = 200, z = 800$

# New Problem with Budget

|         |   A     |    B    |   Capacity   |
|  :---:  | :-----: | :----:  | :---------:  |
| Labour  |   3     |   5     |     $c_1$    |
| Machine |   7     |   4     |     $c_2$    |
| Profit  |   10    |   13    |       -      |

- Single unit of Labour is 5 (USD/hours)
- Single unit of machine is 18 (USD/hours)
- Total budget is 1000 USD

$$
\begin{aligned}
\max z & = 10x_1 + 13x_2 \\
\text{Subject to:} \\
& 3x_1 + 5x_2 \le c_1 \\
& 7x_1 + 4x_2 \le c_2 \\
& 5c_1 + 18c_2 \le 1000 \\
& x_1, x_2, c_1, c_2 \ge 0
\end{aligned}
$$

In [1]:
using JuMP, HiGHS

In [4]:
begin
    m = Model(HiGHS.Optimizer)
    MOI.set(m, MOI.Silent(), true)
    @variable(m, x1 >= 0)
    @variable(m, x2 >= 0)
    @variable(m, c1 >= 0)
    @variable(m, c2 >= 0)
    @objective(m, Max, 10*x1 + 13*x2)
    @constraint(m, 3*x1 + 5*x2 <= c1)
    @constraint(m, 7*x1 + 4*x2 <= c2)
    @constraint(m, 5*c1 + 18*c2 <= 1000)
    optimize!(m)
end

In [5]:
value.([x1, x2])

2-element Vector{Float64}:
  0.0
 10.309278350515465

In [6]:
value.([c1, c2])

2-element Vector{Float64}:
 51.54639175257732
 41.23711340206186

In [7]:
objective_value(m)

134.02061855670104

# Three Products with Constraint Budget

|         |   A     |    B    |   Capacity   |
|  :---:  | :-----: | :----:  | :---------:  |
| Labour  |   3     |   5     |     $c_1$    |
| Machine |   7     |   4     |     $c_2$    |
| Raw Metarial | 3  |   11    |     $c_3$    |
| Profit  |   10    |   13    |       -      |

- Single unit of Labour is 5 (USD/hours)
- Single unit of machine is 18 (USD/hours)
- Single unit of Raw Material is 2 (USD / kgs)
- Total budget is 1000 USD

In [11]:
using Pkg; Pkg.activate("."); using JuMP, HiGHS;

  Activating project at `~/code/julia/notebooks`


In [6]:
begin
    m = Model(HiGHS.Optimizer)
    MOI.set(m, MOI.Silent(), true)
    @variable(m, x1 >= 0)
    @variable(m, x2 >= 0)
    @variable(m, c1 >= 0)
    @variable(m, c2 >= 0)
    @variable(m, c3 >= 0)
    @objective(m, Max, 10*x1 + 13*x2)
    @constraint(m, 3*x1 + 5*x2 <= c1)
    @constraint(m, 7*x1 + 4*x2 <= c2)
    @constraint(m, 3*x1 + 11*x2 <= c3)
    @constraint(m, 5*c1 + 18*c2 + 2*c3 <= 1000)
    optimize!(m)
end

In [7]:
value.([x1, x2])

2-element Vector{Float64}:
 0.0
 8.403361344537815

In [8]:
value.([c1, c2, c3])

3-element Vector{Float64}:
 42.016806722689076
 33.61344537815126
 92.43697478991596

In [9]:
objective_value(m)

109.2436974789916

In [10]:
5 * value(c1) + 18 * value(c2) + 2 * value(c3)

1000.0

In [12]:
JuMP.latex_formulation(m)

$$ \begin{aligned}
\max\quad & 10 x1 + 13 x2\\
\text{Subject to} \quad & 3 x1 + 5 x2 - c1 \leq 0\\
 & 7 x1 + 4 x2 - c2 \leq 0\\
 & 3 x1 + 11 x2 - c3 \leq 0\\
 & 5 c1 + 18 c2 + 2 c3 \leq 1000\\
 & x1 \geq 0\\
 & x2 \geq 0\\
 & c1 \geq 0\\
 & c2 \geq 0\\
 & c3 \geq 0\\
\end{aligned} $$